In [1]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q #

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.8/413.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 13.3 

In [2]:
from transformers import pipeline, set_seed
# Imports the pipeline and set_seed functions from the transformers library.

import matplotlib.pyplot as plt
# Imports the pyplot module from the matplotlib library for plotting.

import pandas as pd
# Imports the pandas library for data manipulation and analysis.

from datasets import load_dataset, load_metric
# Imports functions to load datasets and metrics from the datasets library.

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
# Imports the AutoModelForSeq2SeqLM and AutoTokenizer classes from transformers for sequence-to-sequence models.

import pandas as pd
# Re-imports the pandas library (this line is redundant).

import numpy as np
# Imports the numpy library for numerical operations.

import nltk
# Imports the Natural Language Toolkit (nltk) library for natural language processing tasks.

from nltk.tokenize import sent_tokenize
# Imports the sent_tokenize function from nltk for sentence tokenization.

nltk.download("punkt")
# Downloads the 'punkt' tokenizer models used by nltk for tokenization.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from datasets import load_dataset
# Imports the load_dataset function from the datasets library.

dataset = load_dataset("cnn_dailymail", "3.0.0")
# Loads the CNN/DailyMail dataset with version 3.0.0 and assigns it to the variable 'dataset'.

print(f"Features in cnn_dailymail: {dataset['train'].features}")
# Prints the features of the training split of the CNN/DailyMail dataset.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Features in cnn_dailymail: {'article': Value(dtype='string', id=None), 'highlights': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None)}


In [4]:
# convert dictionary to dataframe for visualisation
dataset_df = pd.DataFrame(dataset['train'])
dataset_df.head()

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [5]:
sample = dataset["train"][1]
# Retrieves the second sample (index 1) from the training split of the CNN/DailyMail dataset and assigns it to the variable 'sample'.

print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
""")
# Prints a formatted string indicating the length of the article.

print(sample["article"][:500])
# Prints the first 500 characters of the article from the sample.

print(f'\nSummary (length: {len(sample["highlights"])}):')
# Prints a formatted string indicating the length of the summary.

print(sample["highlights"])
# Prints the summary from the sample.



Article (excerpt of 500 characters, total length: 4051):

Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


-------------------

## Text Summarization Pipelines


In [6]:
sample_text = dataset["train"][1]["article"][:1000]
# Retrieves the first 1000 characters of the article from the second sample (index 1) in the training split of the dataset and assigns it to the variable 'sample_text'.

summaries = {}
# Initializes an empty dictionary to store the generated summaries from different models.


### Summarization Baseline


In [7]:
def baseline_summary_three_sent(text):
    # Defines a function named baseline_summary_three_sent that takes a single argument 'text'.

    return "\n".join(sent_tokenize(text)[:3])
    # Tokenizes the input 'text' into sentences, takes the first three sentences, joins them with newline characters, and returns the result.


In [8]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)
# Calls the function baseline_summary_three_sent with 'sample_text' as the argument and stores the result in the 'baseline' key of the 'summaries' dictionary.

summaries['baseline']
# Retrieves and outputs the value stored in the 'baseline' key of the 'summaries' dictionary.


'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'

In [9]:
from transformers import pipeline, set_seed
# Imports the pipeline and set_seed functions from the transformers library.

set_seed(42)
# Sets the seed for reproducibility of the model.

pipe = pipeline('text-generation', model='gpt2-medium')
# Creates a text generation pipeline using the GPT-2 medium-sized model.

gpt2_query = sample_text + "\nTL;DR:\n"
# Combines the sample text with a TL;DR (Too Long; Didn't Read) marker.

pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
# Generates text using the GPT-2 model with the provided input and parameters.


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [10]:
pipe_out

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [11]:
pipe_out[0]["generated_text"]
# Accesses the generated text from the first (and only) output of the GPT-2 model.

pipe_out[0]["generated_text"][len(gpt2_query):]
# Slices the generated text to exclude the initial part (up to the length of 'gpt2_query'), which includes the input text and the TL;DR marker.


'To get to the jail that holds the mentally ill, visit our Behind the Scenes blog -- Click here The story doesn\'t end there:\xa0 In 2014, a judge ordered the jail to provide treatment for 40 mental-health detainees in the mental-health unit, as part of a $22,000 settlement.\nInmates in the mental-health unit at Miami-Dade County\'s jail are often locked in a cell that\'s usually just like any other in the facility and can become chaotic. Mental health unit employees are often required to leave the jail and drive to the facility, instead of coming in to work with the inmates themselves. Most mental-health detainees spend hours a day sleeping in front of the wall by the pool, waiting for treatment to show up.\nThere are two more stories from the Inside the Tincup Jail series:\n\xa0\xa0\xa0 The "no contact" policy:\xa0 At an overcrowded state mental health facility in Miami-Dade County, the rules allow the police to try to convince the jail staff to allow a person they believe is mentall

In [12]:
# Assign the generated summary from the GPT-2 model to the 'gpt2' key in the summaries dictionary
# Join the generated text, excluding the input text and TL;DR marker, into a single string with newline characters as separators
summaries['gpt2'] = "\n".join(sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query):]))


In [13]:
pipe = pipeline('summarization', model='t5-small')
# Creates a text summarization pipeline using the T5-small model.

pipe_out = pipe(sample_text)
# Generates a summary for the 'sample_text' using the T5-small model and stores the result in 'pipe_out'.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [14]:
pipe_out

[{'summary_text': "inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court . most often, they face drug charges or charges of assaulting an officer . mentally ill people become more paranoid, delusional, and less likely to follow dir ."}]

In [15]:
# Add the generated summary from the T5-small model to the 't5' key in the summaries dictionary
# Tokenize the generated summary text into sentences and join them into a single string with newline characters as separators
summaries['t5'] = '\n'.join(sent_tokenize(pipe_out[0]['summary_text']))


In [16]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
# Creates a text summarization pipeline using the BART-large-CNN model from the Hugging Face model hub.

pipe_out = pipe(sample_text)
# Generates a summary for the 'sample_text' using the BART-large-CNN model and stores the result in 'pipe_out'.


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [17]:
pipe_out

[{'summary_text': 'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated. Most often, they face drug charges or charges of assaulting an officer. Judge Steven Leifman says the arrests often result from confrontations with police.'}]

In [18]:
# Add the generated summary from the BART-large-CNN model to the 'bart' key in the summaries dictionary
# Tokenize the generated summary text into sentences and join them into a single string with newline characters as separators
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))


In [19]:
summaries["bart"]

'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated.\nMost often, they face drug charges or charges of assaulting an officer.\nJudge Steven Leifman says the arrests often result from confrontations with police.'

In [20]:
pipe = pipeline('summarization', model="google/pegasus-cnn_dailymail")
# Creates a text summarization pipeline using the Pegasus model fine-tuned on the CNN/DailyMail dataset.

pipe_out = pipe(sample_text)
# Generates a summary for the 'sample_text' using the Pegasus model and stores the result in 'pipe_out'.


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [21]:
pipe_out

[{'summary_text': 'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail .<n>Judge Steven Leifman says the charges are usually "avoidable felonies"<n>He says the arrests often result from confrontations with police .<n>Mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]

In [22]:
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [23]:
## Comparing Different Summaries

In [24]:
# Print the heading for the ground truth summary
print("GROUND TRUTH")
# Print the actual summary from the dataset for comparison
print(dataset['train'][1]['highlights'])

# Iterate over each model in the summaries dictionary
for model_name in summaries:
    # Print a separator for clarity
    print("\n\n==================================================================================\n")
    # Print the model name in uppercase as a heading
    print(model_name.upper())
    # Print the generated summary associated with the current model
    print(summaries[model_name])
    # Print a separator for clarity
    print("\n\n==================================================================================\n")


GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .



BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."






GPT2
To get to the jail that holds the mentally ill, visit our Behind the Scenes blog -- Click here The story doesn't end there:  In 2014, a judge ordered the jail to provide treatment for 40 mental-health detainees 

In [25]:
from datasets import load_metric
# Imports the load_metric function from the datasets library.

bleu_metric = load_metric("sacrebleu")
# Loads the BLEU metric for evaluating machine translation outputs, using the SacreBLEU implementation, and assigns it to the variable 'bleu_metric'.


<ipython-input-25-a8047cfa3ed4>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [26]:
# Add the prediction (generated summary from the Pegasus model) and its reference (ground truth summary) to the BLEU metric
bleu_metric.add(prediction=[summaries["pegasus"]], reference=[dataset['train'][1]['highlights']])

# Compute the BLEU score using the added predictions and references, with smoothing method 'floor' and smooth value 0
results = bleu_metric.compute(smooth_method='floor', smooth_value=0)

# Round the precision values to two decimal places and store them in a new key 'precision' in the results dictionary
results['precision'] = [np.round(p, 2) for p in results['precisions']]

# Convert the results dictionary to a pandas DataFrame for better visualization
pd.DataFrame.from_dict(results, orient='index', columns=['Value'])


,Value
score,18.73841
counts,"[27, 14, 10, 6]"
totals,"[67, 66, 65, 64]"
precisions,"[40.298507462686565, 21.21212121212121, 15.384..."
bp,1.0
sys_len,67
ref_len,57
precision,"[40.3, 21.21, 15.38, 9.38]"


In [27]:
rouge_metric = load_metric('rouge')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [28]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
# Define the names of the ROUGE metrics to be computed.

reference = dataset['train'][1]['highlights']
# Get the ground truth summary from the dataset to use as a reference for ROUGE evaluation.

records = []
# Initialize an empty list to store the ROUGE scores for each model.

for model_name in summaries:
    # Iterate over each model in the summaries dictionary.

    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    # Add the generated summary for the current model and the reference summary to the ROUGE metric.

    score = rouge_metric.compute()
    # Compute the ROUGE scores for the current model.

    rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
    # Extract the ROUGE scores for the current model and create a dictionary.

    print('rouge_dict', rouge_dict)
    # Print the ROUGE scores for the current model.

    records.append(rouge_dict)
    # Append the ROUGE scores dictionary to the records list.

pd.DataFrame.from_records(records, index=summaries.keys())
# Convert the records list of dictionaries to a pandas DataFrame with model names as index.


rouge_dict {'rouge1': 0.365079365079365, 'rouge2': 0.14516129032258066, 'rougeL': 0.20634920634920634, 'rougeLsum': 0.2857142857142857}
rouge_dict {'rouge1': 0.15789473684210525, 'rouge2': 0.026490066225165563, 'rougeL': 0.1118421052631579, 'rougeLsum': 0.14473684210526316}
rouge_dict {'rouge1': 0.19780219780219782, 'rouge2': 0.022471910112359546, 'rougeL': 0.13186813186813187, 'rougeLsum': 0.19780219780219782}
rouge_dict {'rouge1': 0.3655913978494624, 'rouge2': 0.13186813186813184, 'rougeL': 0.2150537634408602, 'rougeLsum': 0.3225806451612903}
rouge_dict {'rouge1': 0.5, 'rouge2': 0.24489795918367346, 'rougeL': 0.36000000000000004, 'rougeLsum': 0.46}


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.157895,0.026490,0.111842,0.144737
t5,0.197802,0.022472,0.131868,0.197802
bart,0.365591,0.131868,0.215054,0.322581
pegasus,0.500000,0.244898,0.360000,0.460000


# Evaluationg on the TEST set of the CNN/DailyMail Dataset

In [29]:
def calculate_metric_on_baseline_test_ds(dataset, metric, column_text='article', column_summary='highlights'):
    """
    This function calculates a specified metric on a baseline test dataset for a Natural Language Processing (NLP) task.
    It assumes the task is a text summarization task, where the goal is to generate a summary (e.g., highlights) from a text (e.g., article).

    Parameters:
    dataset (pandas.DataFrame): The test dataset. It should contain a column for the text and a column for the true summary.
    metric (datasets.Metric): The metric to calculate. This should be a metric object from the Hugging Face datasets library.
    column_text (str, optional): The name of the column in the dataset that contains the text. Defaults to 'article'.
    column_summary (str, optional): The name of the column in the dataset that contains the true summary. Defaults to 'highlights'.

    Returns:
    score (float): The calculated score of the metric on the test dataset.
    """
    # Generate summaries for each text in the dataset using a baseline method
    summaries = [baseline_summary_three_sent(text) for text in dataset[column_text]]

    # Add the generated summaries and the true summaries from the dataset to the metric
    metric.add_batch(predictions=summaries, references=dataset[column_summary])

    # Compute the metric score
    score = metric.compute()
    return score


In [30]:
# Sample 1000 examples from the training dataset and shuffle them using a seed for reproducibility
test_sampled = dataset['train'].shuffle(seed=42).select(range(1000))

# Calculate the ROUGE score for the baseline model using the sampled test dataset
score = calculate_metric_on_baseline_test_ds(test_sampled, rouge_metric)

# Extract the ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L, ROUGE-Lsum) from the computed score
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

# Create a DataFrame from the ROUGE scores for the baseline model
# Transpose the DataFrame to have metrics as rows and models as columns
pd.DataFrame.from_dict(rouge_dict, orient='index', columns=['baseline']).T


,rouge1,rouge2,rougeL,rougeLsum
baseline,0.253995,0.100642,0.165754,0.231571


## Strategy to calculate the ROUGE Metric on test dataset for the other Models

In [31]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements.

    Generator function to yield successive batch-sized chunks from list_of_elements.

    Parameters:
    list_of_elements (list): List of elements to be divided into chunks.
    batch_size (int): The size of each chunk.

    Yields:
    list: Batch-sized chunk from list_of_elements.

    """
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    """
    Function to calculate a specified metric on a test dataset for a Natural Language Processing (NLP) task.
    It assumes the task is a text summarization task, where the goal is to generate a summary from a text.

    Parameters:
    dataset (pandas.DataFrame): The test dataset. It should contain a column for the text and a column for the true summary.
    metric (datasets.Metric): The metric to calculate. This should be a metric object from the Hugging Face datasets library.
    model (transformers.PreTrainedModel): The transformer model to use for text generation.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer corresponding to the model.
    batch_size (int, optional): The size of the batches to use for processing. Defaults to 16.
    device (str, optional): The device to run the model on. Defaults to the output of torch.cuda.is_available().
    column_text (str, optional): The name of the column in the dataset that contains the text. Defaults to 'article'.
    column_summary (str, optional): The name of the column in the dataset that contains the true summary. Defaults to 'highlights'.

    Returns:
    score (float): The calculated score of the metric on the test dataset.
    """
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the <n> token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [32]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"

# Load the Pegasus model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

# Calculate ROUGE scores for the Pegasus model on the sampled test dataset
score = calculate_metric_on_test_ds(test_sampled, rouge_metric, model_pegasus, tokenizer, batch_size=8)

# Extract the ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L, ROUGE-Lsum) from the computed score
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

# Create a DataFrame from the ROUGE scores for the Pegasus model
pd.DataFrame(rouge_dict, index=["pegasus"])


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 125/125 [24:05<00:00, 11.57s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.47518,0.278443,0.37406,0.428993


In [33]:
!pip install gradio transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 13.0 MB

In [ ]:
import gradio as gr
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Define the models and their names
models = {
    "Baseline": "baseline_summary_three_sent",
    "GPT-2": "gpt2",
    "T5": "t5",
    "BART": "bart",
    "Pegasus": "pegasus"
}

# Load the Pegasus model and tokenizer
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)

# Define a function to generate summaries using the specified model
def generate_summary(text, model_name):
    model_function = models[model_name]
    if model_function == "pegasus":
        return pipeline("summarization", model=model_ckpt)(text, max_length=128, clean_up_tokenization_spaces=True)[0]['summary_text']
    else:
        return globals()[model_function](text)

# Create a Gradio interface
inputs = gr.Textbox(lines=10, label="Enter the text to summarize")
outputs = [gr.Textbox(label=model_name) for model_name in models.keys()]

gr.Interface(
    fn=generate_summary,
    inputs=inputs,
    outputs=outputs,
    title="Text Summarization with Different Models",
    description="Enter the text and select the model to generate summaries."
).launch()
